In [1]:
# PI, VI

In [2]:
import warnings ; warnings.filterwarnings('ignore')

import gym, gym_walk, gym_aima
import numpy as np
from pprint import pprint
from tqdm import tqdm_notebook as tqdm

from itertools import cycle

import random

np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)

In [3]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [4]:
def print_state_value_function(V, P, n_cols=4, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{:.2f}'.format(v).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [5]:
def probability_success(env, pi, goal_state, iterations=1000):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for i in range(iterations):
        state, done = env.reset(), False
        while not done:
            state, _, done, _ = env.step(pi(state))
        results.append(state == goal_state)
    return np.sum(results)/len(results)

In [6]:
def mean_return(env, pi, iterations=1000):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for i in range(iterations):
        state, done = env.reset(), False
        results.append(0.0)
        while not done:
            state, reward, done, _ = env.step(pi(state))
            results[-1] += reward
    return np.mean(results)

# Slippery Walk Five MDP and sample policy

In [7]:
env = gym.make('SlipperyWalkFive-v0')
P = env.env.P

WEST, EAST = range(2)
pi = lambda s: {
    0:WEST, 1:WEST, 2:WEST, 3:WEST, 4:WEST, 5:WEST, 6:WEST
}[s]
print_policy(pi, P, action_symbols=('<', '>'), n_cols=7)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi, goal_state=6)*100, 
    mean_return(env, pi)))

Policy:
|           | 01      < | 02      < | 03      < | 04      < | 05      < |           |
Reaches goal 4.90%. Obtains an average undiscounted return of 0.0490.


# Policy Evaluation

In [8]:
def policy_evaluation(pi, P, gamma=1.0, theta=1e-10):
    prev_V = np.zeros(len(P))
    while True:
        V = np.zeros(len(P))
        for s in range(len(P)):
            for prob, next_state, reward, done in P[s][pi(s)]:
                V[s] += prob * (reward + gamma * prev_V[next_state] * (not done))
        if np.max(np.abs(prev_V - V)) < theta:
            break
        prev_V = V.copy()
    return V

In [9]:
V = policy_evaluation(pi, P)
print_state_value_function(V, P, n_cols=7)

State-value function:
|           | 01   0.00 | 02   0.01 | 03   0.04 | 04   0.11 | 05   0.33 |           |


# Policy Improvement

In [10]:
def policy_improvement(V, P, gamma=1.0):
    Q = np.zeros((len(P), len(P[0])), dtype=np.float64)
    for s in range(len(P)):
        for a in range(len(P[s])):
            for prob, next_state, reward, done in P[s][a]:
                Q[s][a] += prob * (reward + gamma * V[next_state] * (not done))
    new_pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]
    return new_pi

In [11]:
improved_pi = policy_improvement(V, P)
print_policy(improved_pi, P, action_symbols=('<', '>'), n_cols=7)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, improved_pi, goal_state=6)*100, 
    mean_return(env, improved_pi)))

Policy:
|           | 01      > | 02      > | 03      > | 04      > | 05      > |           |
Reaches goal 95.10%. Obtains an average undiscounted return of 0.9510.


In [12]:
# how about we evaluate the improved policy?
improved_V = policy_evaluation(improved_pi, P)
print_state_value_function(improved_V, P, n_cols=7)

State-value function:
|           | 01   0.67 | 02   0.89 | 03   0.96 | 04   0.99 | 05   1.00 |           |


In [13]:
# can we improved the improved policy?
improved_improved_pi = policy_improvement(improved_V, P)
print_policy(improved_improved_pi, P, action_symbols=('<', '>'), n_cols=7)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, improved_improved_pi, goal_state=6)*100, 
    mean_return(env, improved_improved_pi)))

Policy:
|           | 01      > | 02      > | 03      > | 04      > | 05      > |           |
Reaches goal 95.10%. Obtains an average undiscounted return of 0.9510.


In [14]:
# it is the same policy
# if we evaluate again, we can see there is nothing to improve 
# that also means we reached the optimal policy
improved_improved_V = policy_evaluation(improved_improved_pi, P)
print_state_value_function(improved_improved_V, P, n_cols=7)

State-value function:
|           | 01   0.67 | 02   0.89 | 03   0.96 | 04   0.99 | 05   1.00 |           |


In [15]:
# state-value function didn't improve, then we reach the optimal policy
np.all(improved_V == improved_improved_V)

True

# Policy Iteration

In [16]:
def policy_iteration(P, gamma=1.0):
    random_actions = np.random.choice(tuple(P[0].keys()), len(P))
    pi = lambda s: {s:a for s, a in enumerate(random_actions)}[s]
    while True:
        old_pi = {s:pi(s) for s in range(len(P))}
        V = policy_evaluation(pi, P, gamma)
        pi = policy_improvement(V, P, gamma)
        if old_pi == {s:pi(s) for s in range(len(P))}:
            break
    return V, pi

In [17]:
optimal_V, optimal_pi = policy_iteration(P)
print('Optimal policy and state-value function (PI):')
print_policy(optimal_pi, P, action_symbols=('<', '>'), n_cols=7)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, optimal_pi, goal_state=6)*100, 
    mean_return(env, optimal_pi)))
print()
print_state_value_function(optimal_V, P, n_cols=7)

Optimal policy and state-value function (PI):
Policy:
|           | 01      > | 02      > | 03      > | 04      > | 05      > |           |
Reaches goal 95.10%. Obtains an average undiscounted return of 0.9510.

State-value function:
|           | 01   0.67 | 02   0.89 | 03   0.96 | 04   0.99 | 05   1.00 |           |


In [18]:
np.all(improved_V == optimal_V)

True

# Frozen Lake MDP and sample policy

In [19]:
env = gym.make('FrozenLake-v0')
P = env.env.P

WEST, SOUTH, EAST, NORTH = range(4)
pi = lambda s: {
    0:EAST, 1:WEST, 2:SOUTH, 3:NORTH,
    4:WEST, 5:WEST, 6:EAST, 7:WEST,
    8:NORTH, 9:SOUTH, 10:NORTH, 11:WEST,
    12:WEST, 13:EAST, 14:SOUTH, 15:WEST
}[s]
print_policy(pi, P)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi, goal_state=15)*100, 
    mean_return(env, pi)))

Policy:
| 00      > | 01      < | 02      v | 03      ^ |
| 04      < |           | 06      > |           |
| 08      ^ | 09      v | 10      ^ |           |
|           | 13      > | 14      v |           |
Reaches goal 11.50%. Obtains an average undiscounted return of 0.1150.


# Policy Evaluation

In [20]:
V = policy_evaluation(pi, P, gamma=0.99)
print_state_value_function(V, P)

State-value function:
| 00   0.10 | 01   0.05 | 02   0.05 | 03   0.05 |
| 04   0.15 |           | 06   0.05 |           |
| 08   0.20 | 09   0.26 | 10   0.10 |           |
|           | 13   0.50 | 14   0.74 |           |


# Policy Improvement

In [21]:
new_pi = policy_improvement(V, P, gamma=0.99)
print_policy(new_pi, P)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, new_pi, goal_state=15)*100, 
    mean_return(env, new_pi)))

Policy:
| 00      < | 01      ^ | 02      < | 03      ^ |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |
Reaches goal 72.00%. Obtains an average undiscounted return of 0.7200.


In [22]:
new_V = policy_evaluation(new_pi, P, gamma=0.99)
print_state_value_function(new_V, P)

State-value function:
| 00   0.53 | 01   0.45 | 02   0.38 | 03   0.37 |
| 04   0.55 |           | 06   0.32 |           |
| 08   0.58 | 09   0.63 | 10   0.60 |           |
|           | 13   0.73 | 14   0.86 |           |


# Policy Iteration

In [23]:
V_best_p, pi_best_p = policy_iteration(P, gamma=0.99)
print('Optimal policy and state-value function (PI):')
print_policy(pi_best_p, P)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_best_p, goal_state=15)*100, 
    mean_return(env, pi_best_p)))
print()
print_state_value_function(V_best_p, P)

Optimal policy and state-value function (PI):
Policy:
| 00      < | 01      ^ | 02      ^ | 03      ^ |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |
Reaches goal 73.20%. Obtains an average undiscounted return of 0.7320.

State-value function:
| 00   0.54 | 01   0.50 | 02   0.47 | 03   0.46 |
| 04   0.56 |           | 06   0.36 |           |
| 08   0.59 | 09   0.64 | 10   0.62 |           |
|           | 13   0.74 | 14   0.86 |           |


In [24]:
print('For comparison, original policy and state-value function:')
print_policy(pi, P)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi, goal_state=15)*100, 
    mean_return(env, pi)))
print()
print_state_value_function(V, P)

For comparison, original policy and state-value function:
Policy:
| 00      > | 01      < | 02      v | 03      ^ |
| 04      < |           | 06      > |           |
| 08      ^ | 09      v | 10      ^ |           |
|           | 13      > | 14      v |           |
Reaches goal 11.50%. Obtains an average undiscounted return of 0.1150.

State-value function:
| 00   0.10 | 01   0.05 | 02   0.05 | 03   0.05 |
| 04   0.15 |           | 06   0.05 |           |
| 08   0.20 | 09   0.26 | 10   0.10 |           |
|           | 13   0.50 | 14   0.74 |           |


In [25]:
print('For comparison, improved policy and state-value function:')
print_policy(new_pi, P)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, new_pi, goal_state=15)*100, 
    mean_return(env, new_pi)))
print()
print_state_value_function(new_V, P)

For comparison, improved policy and state-value function:
Policy:
| 00      < | 01      ^ | 02      < | 03      ^ |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |
Reaches goal 72.00%. Obtains an average undiscounted return of 0.7200.

State-value function:
| 00   0.53 | 01   0.45 | 02   0.38 | 03   0.37 |
| 04   0.55 |           | 06   0.32 |           |
| 08   0.58 | 09   0.63 | 10   0.60 |           |
|           | 13   0.73 | 14   0.86 |           |


# Slippery Walk Five

In [26]:
env = gym.make('SlipperyWalkFive-v0')
P = env.env.P

# Value Iteration

In [27]:
def value_iteration(P, gamma=1.0, theta=1e-10):
    V = np.zeros(len(P), dtype=np.float64)
    while True:
        Q = np.zeros((len(P), len(P[0])), dtype=np.float64)
        for s in range(len(P)):
            for a in range(len(P[s])):
                for prob, next_state, reward, done in P[s][a]:
                    Q[s][a] += prob * (reward + gamma * V[next_state] * (not done))
        if np.max(np.abs(V - np.max(Q, axis=1))) < theta:
            break
        V = np.max(Q, axis=1)
    pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]
    return V, pi

In [28]:
optimal_V, optimal_pi = value_iteration(P)
print('Optimal policy and state-value function (PI):')
print_policy(optimal_pi, P, action_symbols=('<', '>'), n_cols=7)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, optimal_pi, goal_state=6)*100, 
    mean_return(env, optimal_pi)))
print()
print_state_value_function(optimal_V, P, n_cols=7)
# |            | 01   0.668 | 02   0.890 | 03   0.964 | 04   0.989 | 05   0.997 |            |

Optimal policy and state-value function (PI):
Policy:
|           | 01      > | 02      > | 03      > | 04      > | 05      > |           |
Reaches goal 95.10%. Obtains an average undiscounted return of 0.9510.

State-value function:
|           | 01   0.67 | 02   0.89 | 03   0.96 | 04   0.99 | 05   1.00 |           |


# Frozen Lake MDP

In [29]:
env = gym.make('FrozenLake-v0')
P = env.env.P

In [30]:
V_best_v, pi_best_v = value_iteration(P, gamma=0.99)
print('Optimal policy and state-value function (VI):')
print_policy(pi_best_v, P)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_best_v, goal_state=15)*100, 
    mean_return(env, pi_best_v)))
print()
print_state_value_function(V_best_v, P)

Optimal policy and state-value function (VI):
Policy:
| 00      < | 01      ^ | 02      ^ | 03      ^ |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |
Reaches goal 73.20%. Obtains an average undiscounted return of 0.7320.

State-value function:
| 00   0.54 | 01   0.50 | 02   0.47 | 03   0.46 |
| 04   0.56 |           | 06   0.36 |           |
| 08   0.59 | 09   0.64 | 10   0.62 |           |
|           | 13   0.74 | 14   0.86 |           |


In [31]:
print('For comparison, optimal policy and state-value function (PI):')
print_policy(pi_best_p, P)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_best_p, goal_state=15)*100, 
    mean_return(env, pi_best_p)))
print()
print_state_value_function(V_best_p, P)

For comparison, optimal policy and state-value function (PI):
Policy:
| 00      < | 01      ^ | 02      ^ | 03      ^ |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |
Reaches goal 73.20%. Obtains an average undiscounted return of 0.7320.

State-value function:
| 00   0.54 | 01   0.50 | 02   0.47 | 03   0.46 |
| 04   0.56 |           | 06   0.36 |           |
| 08   0.59 | 09   0.64 | 10   0.62 |           |
|           | 13   0.74 | 14   0.86 |           |


# Changing the Frozen Lake environment MDP

In [32]:
env = gym.make('FrozenLake-v0')
P = env.env.P

# change reward function
reward_goal, reward_holes, reward_others = 1, -1, -0.01
goal, hole = 15, [5, 7, 11, 12]
for s in range(len(P)):
    for a in range(len(P[s])):
        for t in range(len(P[s][a])):
            values = list(P[s][a][t])
            if values[1] == goal:
                values[2] = reward_goal
                values[3] = False
            elif values[1] in hole:
                values[2] = reward_holes
                values[3] = False
            else:
                values[2] = reward_others
                values[3] = False
            if s in hole or s == goal:
                values[2] = 0
                values[3] = True
            P[s][a][t] = tuple(values)

# change transition function
prob_action, prob_drift_one, prob_drift_two = 0.8, 0.1, 0.1
for s in range(len(P)):
    for a in range(len(P[s])):
        for t in range(len(P[s][a])):
            if P[s][a][t][0] == 1.0:
                continue
            values = list(P[s][a][t])
            if t == 0:
                values[0] = prob_drift_one
            elif t == 1:
                values[0] = prob_action
            elif t == 2:
                values[0] = prob_drift_two
            P[s][a][t] = tuple(values)

env.env.P = P

In [33]:
V_best, pi_best = policy_iteration(env.env.P, gamma=0.99)
print('Optimal policy and state-value function (PI):')
print_policy(pi_best, P)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_best, goal_state=15)*100, 
    mean_return(env, pi_best)))
print()
print_state_value_function(V_best, P)

Optimal policy and state-value function (PI):
Policy:
| 00      v | 01      ^ | 02      v | 03      ^ |
| 04      < |           | 06      v |           |
| 08      > | 09      v | 10      < |           |
|           | 13      > | 14      > |           |
Reaches goal 85.20%. Obtains an average undiscounted return of 0.5281.

State-value function:
| 00   0.43 | 01   0.35 | 02   0.41 | 03   0.28 |
| 04   0.46 |           | 06   0.45 |           |
| 08   0.64 | 09   0.88 | 10   0.83 |           |
|           | 13   0.94 | 14   0.98 |           |


In [34]:
V_best, pi_best = value_iteration(env.env.P, gamma=0.99)
print('Optimal policy and state-value function (PI):')
print_policy(pi_best, P)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_best, goal_state=15)*100, 
    mean_return(env, pi_best)))
print()
print_state_value_function(V_best, P)

Optimal policy and state-value function (PI):
Policy:
| 00      v | 01      ^ | 02      v | 03      ^ |
| 04      < |           | 06      v |           |
| 08      > | 09      v | 10      < |           |
|           | 13      > | 14      > |           |
Reaches goal 85.20%. Obtains an average undiscounted return of 0.5281.

State-value function:
| 00   0.43 | 01   0.35 | 02   0.41 | 03   0.28 |
| 04   0.46 |           | 06   0.45 |           |
| 08   0.64 | 09   0.88 | 10   0.83 |           |
|           | 13   0.94 | 14   0.98 |           |


# Russell & Norvig's Gridworld

In [35]:
env = gym.make('RussellNorvigGridworld-v0')
P = env.env.P

In [36]:
V_best_p, pi_best = policy_iteration(P)
print('Optimal policy and state-value function (PI):')
print_policy(pi_best, P)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_best, goal_state=3)*100, 
    mean_return(env, pi_best)))
print()
print_state_value_function(V_best_p, P)

Optimal policy and state-value function (PI):
Policy:
| 00      > | 01      > | 02      > |           |
| 04      ^ |           | 06      ^ |           |
| 08      ^ | 09      < | 10      < | 11      < |
Reaches goal 98.80%. Obtains an average undiscounted return of 0.7041.

State-value function:
| 00   0.81 | 01   0.87 | 02   0.92 |           |
| 04   0.76 |           | 06   0.66 |           |
| 08   0.71 | 09   0.66 | 10   0.61 | 11   0.39 |


In [37]:
V_best_v, pi_best = value_iteration(P)
print('Optimal policy and state-value function (PI):')
print_policy(pi_best, P)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_best, goal_state=3)*100, 
    mean_return(env, pi_best)))
print()
print_state_value_function(V_best_v, P)

Optimal policy and state-value function (PI):
Policy:
| 00      > | 01      > | 02      > |           |
| 04      ^ |           | 06      ^ |           |
| 08      ^ | 09      < | 10      < | 11      < |
Reaches goal 98.80%. Obtains an average undiscounted return of 0.7041.

State-value function:
| 00   0.81 | 01   0.87 | 02   0.92 |           |
| 04   0.76 |           | 06   0.66 |           |
| 08   0.71 | 09   0.66 | 10   0.61 | 11   0.39 |


In [38]:
WEST, SOUTH, EAST, NORTH = range(4)
pi = lambda s: {
    0:EAST, 1:EAST, 2:EAST, 3:WEST,
    4:NORTH, 5:WEST, 6:NORTH, 7:WEST,
    8:NORTH, 9:WEST, 10:WEST, 11:WEST
}[s]
print('Re-construct optimal policy:')
print_policy(pi, P)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi, goal_state=3)*100, 
    mean_return(env, pi)))

Re-construct optimal policy:
Policy:
| 00      > | 01      > | 02      > |           |
| 04      ^ |           | 06      ^ |           |
| 08      ^ | 09      < | 10      < | 11      < |
Reaches goal 98.80%. Obtains an average undiscounted return of 0.7041.


In [39]:
V = policy_evaluation(pi, P)
print('Evaluate optimal policy:')
print_state_value_function(V, P)

Evaluate optimal policy:
State-value function:
| 00   0.81 | 01   0.87 | 02   0.92 |           |
| 04   0.76 |           | 06   0.66 |           |
| 08   0.71 | 09   0.66 | 10   0.61 | 11   0.39 |


In [40]:
pi = policy_improvement(V, P)
print('Improve optimal policy (nothing to improve -- is the same policy, because it is optimal):')
print_policy(pi, P)

Improve optimal policy (nothing to improve -- is the same policy, because it is optimal):
Policy:
| 00      > | 01      > | 02      > |           |
| 04      ^ |           | 06      ^ |           |
| 08      ^ | 09      < | 10      < | 11      < |


In [41]:
print('There are no differences, nothing to improve on the optimal policy and state-value function:')
print(np.abs(V_best_p - V))
print(np.abs(V_best_v - V))

There are no differences, nothing to improve on the optimal policy and state-value function:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
